In [20]:
print("Notebook is working")

Notebook is working


## Imports

In [21]:
import requests
from datetime import datetime, time, timezone, date as dt_date

## Stadium and game information

In [22]:
# Braves (Truist Park)
stadium_name = "Truist Park"
latitude = 33.8908
longitude = -84.4678
team_id = 144
date = dt_date.today().isoformat()

In [23]:
# 1. Call MLB schedule API for the Braves (team ID 144) for today's date
mlb_url = "https://statsapi.mlb.com/api/v1/schedule"
mlb_params = {
    "sportId": 1,
    "date": date,     # today's e variable you've set
    "teamId": 144     # Braves team ID
}
mlb_resp = requests.get(mlb_url, params=mlb_params)
mlb_data = mlb_resp.json()

games = mlb_data.get("dates", [])


In [24]:
if games:
    game = games[0]["games"][0]  # First Braves game today
    utc_start = game["gameDate"]  # e.g. "2025-07-01T23:20:00Z"
    print("Raw UTC start time:", utc_start)
else:
    print("No Braves game scheduled today.")


Raw UTC start time: 2025-07-06T15:35:00Z


In [25]:
# Convert UTC time to your local time zone
dt_utc = datetime.fromisoformat(utc_start.replace("Z", "+00:00"))
dt_local = dt_utc.astimezone(tz=None)  # Auto-detects your system's local time
game_hour = dt_local.hour

print("Local start time:", dt_local)
print("Start hour only (for filtering):", game_hour)


Local start time: 2025-07-06 11:35:00-04:00
Start hour only (for filtering): 11


In [26]:
# Open-Meteo API
weather_url = "https://api.open-meteo.com/v1/forecast"
weather_params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": date,
    "end_date": date,
    "hourly": "winddirection_10m,windspeed_10m",
    "timezone": "America/New_York"
}

weather_resp = requests.get(weather_url, params=weather_params)
weather_data = weather_resp.json()


In [27]:
def filter_for_game_time(data, start_hour, end_hour):
    hourly = data["hourly"]
    times = hourly["time"]
    directions = hourly["winddirection_10m"]
    speeds_kmh = hourly["windspeed_10m"]

    filtered = []

    for i, t_str in enumerate(times):
        dt = datetime.strptime(t_str, "%Y-%m-%dT%H:%M")
        if start_hour <= dt.hour <= end_hour:
            filtered.append({
                "time": t_str,
                "direction_deg": directions[i],
                "speed_kmh": speeds_kmh[i],
                "speed_mph": round(speeds_kmh[i] * 0.621371, 2)
            })

    return filtered


In [31]:
if game_hour is not None:
    filtered_wind = filter_for_game_time(weather_data, start_hour=game_hour, end_hour=game_hour+3)

    for entry in filtered_wind:
        cardinal = degrees_to_cardinal(entry["direction_deg"])
        print(f"Time: {entry['time']}, Direction: {entry['direction_deg']}° ({cardinal}), "
          f"Speed: {entry['speed_kmh']} km/h ({entry['speed_mph']} mph)")

else:
    print("No game today — no weather filtering.")


Time: 2025-07-06T11:00, Direction: 124° (SE), Speed: 1.3 km/h (0.81 mph)
Time: 2025-07-06T12:00, Direction: 305° (NW), Speed: 7.5 km/h (4.66 mph)
Time: 2025-07-06T13:00, Direction: 308° (NW), Speed: 8.7 km/h (5.41 mph)
Time: 2025-07-06T14:00, Direction: 319° (NW), Speed: 10.5 km/h (6.52 mph)


In [29]:
def degrees_to_cardinal(degrees):
    dirs = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    ix = round(degrees / 45) % 8
    return dirs[ix]